In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import hamhelper.plotting as hp
import glob as glob
from Mink import local_max
import hamhelper.plotting as hp


# Print working dir from os
import os
print(os.getcwd())

: 

# Investigating $NbSe_2$ TMD Flakes
 1. TEM and EDX
 2. Laue XRD

In [ ]:
## Osiris TEM - EDX - Import Data
# %matplotlib notebook
data_dir = 'data\\'
edx_data = glob.glob(data_dir + '*EDX*.txt')
print(f"Found {len(edx_data)} EDX files!")

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

known_peaks = {
    'Hg': [1.8, 10, 10.5,11.8],
    'Nb': [2.2,15.2, 16.5],
    'Se': [1.3, 11.2, 12.5],
    'Br': [11.6, 11.8, 12, 12.2],
    'Sn': [2.9],
    'Cu': [1, 8],
    'C': [0.3],
    'O': [0.52],
}

from scipy.optimize import curve_fit

def find_peaks(y, N=20, strict=False, snr_threshold=1, minsep=20):
    """
    Identify peaks in the data while filtering out noise and ensuring minimum separation.

    Parameters:
    - y: The input data array.
    - N: The window size for local maxima detection.
    - strict: Whether to enforce stricter peak detection.
    - snr_threshold: Minimum signal-to-noise ratio for a peak to be considered.
    - minsep: Minimum separation between peaks (in the same units as the index).

    Returns:
    - peak_h_2: Heights of the filtered peaks.
    - peak_idx_2: Indices of the filtered peaks.
    """
    peak_h, peak_idx = local_max(y, N, strict)
    
    # Calculate global noise level (e.g., standard deviation of the baseline)
    noise_level = np.std(y[-N:])  # Assuming the last N points represent baseline noise
    
    # Filter peaks based on SNR
    filtered_peaks = []
    for h, idx in zip(peak_h, peak_idx):
        snr = h / noise_level
        if snr >= snr_threshold:
            filtered_peaks.append((h, idx))
    
    # Sort peaks by height (descending) to prioritize stronger peaks
    filtered_peaks.sort(key=lambda x: x[0], reverse=True)
    
    # Enforce minimum separation
    final_peaks = []
    for h, idx in filtered_peaks:
        if all(abs(idx - other_idx) >= minsep for _, other_idx in final_peaks):
            final_peaks.append((h, idx))
    
    # Separate heights and indices
    peak_h_2, peak_idx_2 = zip(*final_peaks) if final_peaks else ([], [])
    
    return np.array(peak_h_2), np.array(peak_idx_2)

def import_edx(file, plot=True, verbose=True):
    energy, counts = np.genfromtxt(file, skip_header=0, unpack=True)
    box = 2
    counts_smooth = smooth(counts, box)
    peak_h, peak_idx = find_peaks(counts, N=40, strict=False, snr_threshold=35)
    
    if verbose:
        print(f"Imported {file} of shape ({energy.size}, {counts.size}).")
    
    if plot:
        fig, ax = plt.subplots(1, 1, figsize=np.array([3.3*2.8, 2.2]))
        plt.plot(energy / 1000, counts_smooth, 'r-', linewidth=0.8, zorder=3)
        
        dE = 0.1  # Energy tolerance for matching peaks
        peak_array = []
        for idx in peak_idx:
            peak_energy = energy[idx] / 1000
            peak_count = counts_smooth[idx]
            matched = False
            
            # Check if the peak matches any known peak
            for key, value in known_peaks.items():
                for v in value:
                    if abs(peak_energy - v) < dE:
                        matched = True
                        peak_array.append([peak_energy,peak_count,key])
                        # Annotate the peak with the element name
                        plt.annotate(
                            key,
                            (peak_energy, peak_count),
                            textcoords="offset points",
                            xytext=(0, 10),
                            ha='center'
                        )
                        # Draw a stem line from the scatter point to the annotation
                        plt.vlines(
                            x=peak_energy,
                            ymin=-100,
                            ymax=peak_count,
                            colors='b',
                            linestyles='dotted',
                            linewidth=0.8,
                            zorder=2
                        )
                        plt.vlines(
                            x=peak_energy,
                            ymin=-100,
                            ymax=0,
                            colors='b',
                            linestyles='solid',
                            linewidth=0.8,
                            zorder=2
                        )
                        # Add a small blue scatter point
                        plt.scatter(
                            peak_energy,
                            peak_count,
                            c='b',
                            s=10,
                            zorder=4
                        )
                        
                        # Fit gaussian to the peak
                        # Use a small range around the peak for fitting
                        width=50
                        x_data = energy[idx-width:idx+width]
                        y_data = counts_smooth[idx-width:idx+width]
                        # plt.plot(x_data / 1000, y_data, 'y-', linewidth=2, zorder=30)
                        # Fit a Gaussian to the data
                        # Initial guess for the parameters
                        initial_guess = [100, 8.02, 1]
                        try:
                            # plt.plot(model_x, fit_y, 'g-', linewidth=2, zorder=30)
                            print("Found gaussian")
                        except RuntimeError:
                            print(f"Gaussian fit failed for peak at {peak_energy:.2f} keV")
                        break
                if matched:
                    break
        
        plt.xlabel('Energy (keV)')
        plt.ylabel('Counts')
        plt.legend(title=f"Box smoothing: {box}", framealpha=0)
        ax.set(ylim=(-0.05*counts_smooth.max(), 1.1 * counts_smooth.max()))
        hp.despine()
        plt.savefig(f"plots\\raw_data\\{file.split('\\')[-1].replace('.txt', '.pdf')}", dpi=300, transparent=True, bbox_inches='tight')
    
    # sort peak array based on the first index
    peak_array = sorted(peak_array, key=lambda x: x[0])
    return energy, counts, peak_array

edx_dict = {}
for data in edx_data:
    time_str = data.split("\\")[-1].split(" ")[0].replace(".", ":")
    energy, counts, peak_array = import_edx(data, plot=True, verbose=True)
    plt.show()
    edx_dict[time_str] = [energy, counts, peak_array]

    # Get unique atoms
    unique_atoms = list(set(np.array(peak_array)[:, 2]))
    atomic_ratios = {}
    for atom in unique_atoms:
        atomic_ratios[atom] = 0
    
    fig, axs = plt.subplots(1, len(peak_array), figsize=np.array([2*len(peak_array), 3])*0.8, sharey=True)
    for i, (x_peak, y_peak, atom) in enumerate(peak_array):
        # print('Doing E=', x_peak, 'for', atom, 'with peak of', y_peak)
        ax = axs[i]
        dE = 0.1 + 0.05*x_peak
        delimiter = np.where(np.logical_and(energy/1000 >= x_peak-dE, energy/1000 <= x_peak+dE), 1, 0)
        x = energy[np.where(delimiter==1)]
        y = counts[np.where(delimiter==1)]
        
        initial_guess = [y_peak, x_peak*1000, 100, 20]
        def gaussian(x, amp, mean, stddev, bg=0):
            return amp * np.exp(-((x - mean) ** 2) / (2 * stddev ** 2)) + bg
        popt, pcov = curve_fit(gaussian, x, y, p0=initial_guess)
        # print(popt)
        model_x = np.linspace(x.min(), x.max(), 1000)
        fit_y = gaussian(model_x, *popt)
        
        # calculate area under the curve
        area = np.trapz(fit_y-popt[-1], model_x/1000)
        # calculate error of the area
        area_error = np.sqrt(np.sum((np.gradient(fit_y, model_x) * popt[-1])**2))
        print(f"Area under the curve: {area} +/- {area_error}")
        # print(f"Area under the curve: {area}")
        atomic_ratios[atom] += area
        ax.plot(model_x/1000, fit_y, 'g-', linewidth=1, alpha=0.8, zorder=30)
        ax.plot(x/1000, y, 'r-', linewidth=1, zorder=3)
        ax.set(xlabel=atom, title=f'Area: {area:.2f}')
        if i == 0:
            hp.despine(ax)
        else:
            hp.despine(ax, left=False)
    
    plt.savefig(f"plots\\{data.split("\\")[-1][:-4]}.pdf", dpi=300, transparent=True, bbox_inches='tight')
    plt.show()
    print(atomic_ratios)
    
    # Normalize to per number of Nb
    norm = atomic_ratios['Nb']
    for atom in atomic_ratios.keys():
        atomic_ratios[atom] = atomic_ratios[atom] / norm
    print(atomic_ratios)

    # Normalize as atomic percent
    total = sum(atomic_ratios.values())
    for atom in atomic_ratios.keys():
        atomic_ratios[atom] = np.around(atomic_ratios[atom] / total * 100, 1)
    print(atomic_ratios)

In [ ]:
# Osiris TEM - EDX - Analyse Data Data


## Laue Analysis

In [ ]:
dx = 76.5
dy = 82.8
mm_per_px = np.sqrt(dx**2 + dy**2)/1000  # mm/px
min_radius = 100 # px
L = 36.5  # cm to mm

def mm_to_two_theta(mm, d=L, d2=0):
    """
    Convert distance in mm to two theta angle in degrees.
    """
    return np.rad2deg(d2-np.arctan2(mm, d))  # radians -> deg

min_radius = min_radius * mm_per_px  # px to mm

## Import Data from ImageJ
data_dir = 'data\\'
laue_data_1 = glob.glob(data_dir + 'laue2\\data\\data_3.csv')
laue_data_2 = glob.glob(data_dir + 'laue2\\data\\data_small*.csv')
laue_error = glob.glob(data_dir + 'laue2\\errors\\*.csv')
print(f"Found {len(laue_data_1)} Laue data files!")
print(f"Found {len(laue_error)} Laue error files!")
# Radius_[pixels]
# Normalized_Integrated_Intensity
# Minima_from_Integrated_Intensity
# Maxima_from_Integrated_Intensity
# Stdev_from_Integrated_Intensity
# Number_of_counted_pixels_within_a_bin
signal = np.genfromtxt(laue_data_1[0], skip_header=1, unpack=False, delimiter=',', dtype=np.float64,
                                missing_values=["NaN", "nan", ""],
                                filling_values=np.nan)
signal = np.where(np.abs(signal) > 1e20, np.nan, signal)
s_x = signal[:,0]*mm_per_px
s_y = signal[:,1]
s_yerr = signal[:,4]
signal2 = np.genfromtxt(laue_data_2[0], skip_header=1, unpack=False, delimiter=',', dtype=np.float64,
                                missing_values=["NaN", "nan", ""],
                                filling_values=np.nan)
signal2 = np.where(np.abs(signal2) > 1e20, np.nan, signal2)
s_x2 = signal2[:,0]*mm_per_px
s_y2 = signal2[:,1]
s_yerr2 = signal2[:,4]

# Load background
b_x_arr = []
b_y_arr = []
b_yerr_arr = []
for i, file in enumerate(laue_error):
    print(f'Loading background {i+1}/{len(laue_error)}: {file.split("\\")[-1]}')
    background = np.genfromtxt(file, skip_header=1, unpack=False, delimiter=',', dtype=np.float64,
                                missing_values=["NaN", "nan", ""],
                                filling_values=np.nan)
    background = np.where(np.abs(background) > 1e20, np.nan, background)
    # print(background)
    b_x = background[:,0]*mm_per_px
    b_y = background[:,1]
    b_yerr = background[:,4]
    b_x_arr.append(b_x)
    b_y_arr.append(b_y)
    b_yerr_arr.append(b_yerr)
    print(background.shape)
    # print(f'|| mean bg = {np.nanmean(b_y):.2f} +/- {np.nanmean(b_yerr):.2f}')

b_x = np.mean(b_x_arr, axis=0)
b_y = np.mean(b_y_arr, axis=0)
b_yerr = np.nanmean(b_yerr_arr, axis=0)

fig, ax = plt.subplots(1, 1, figsize=np.array([3.3*1.5, 2.2])*1.3)

ax.plot(s_x, s_y, 'r-', linewidth=0.8, zorder=2, alpha=0.8, label='Signal 1')
ax.plot(s_x2, s_y2, 'b-', linewidth=0.8, zorder=2, alpha=0.8, label='Signal 2')
# plot background with error bands
# delimiter = np.where(np.logical_and(b_x != np.nan, b_y != np.nan), 1,)
hp.errorbands(b_x, b_y, b_yerr, color='k', alpha=0.2, zorder=1)
ax.plot(b_x, b_y, 'k-', linewidth=0.8, zorder=1, label='Background')

ax.axvline(x=min_radius, color='k', linestyle='--', linewidth=0.8, zorder=2)

ax.set(xlabel='Dist From Center [mm]', ylabel='Intensity [a.u.]')
plt.legend(framealpha=0)
hp.despine()
plt.savefig(f"plots\\laue_raw.pdf", dpi=300, transparent=True, bbox_inches='tight')
plt.show()

fig, ax = plt.subplots(1, 1, figsize=np.array([3.3*1.5, 2.2])*1.3)

delimiter = np.where(s_x > min_radius, 1, 0)
subtracted_x = s_x[delimiter == 1]
subtracted_y = s_y[delimiter == 1] - b_y[delimiter == 1]
subtracted_b_x = b_x[delimiter == 1]
subtracted_b_yerr = b_y[delimiter == 1]
delimiter = np.where(s_x2 > min_radius, 1, 0)
subtracted_x2 = s_x2[delimiter == 1]
subtracted_y2 = s_y2[delimiter == 1] - b_y[delimiter == 1]

ax.plot(90+mm_to_two_theta(subtracted_x), subtracted_y, 'r-', linewidth=0.8, zorder=2, alpha=0.8, label='Laue 1')
ax.plot(90+mm_to_two_theta(subtracted_x2), subtracted_y2, 'b-', linewidth=0.8, zorder=2, alpha=0.8, label='Laue 2')
# hp.errorbands(mm_to_two_theta(subtracted_b_x), 0, subtracted_b_yerr, color='white', alpha=0.5, zorder=3)

ax.set(xlabel=r"$2\theta$ [deg]", ylabel='Intensity [a.u.]', ylim=[-20, 1.1 * np.nanmax(s_y - b_y)])
plt.legend(framealpha=0)
hp.despine()
plt.savefig(f"plots\\laue_cleaned.pdf", dpi=300, transparent=True, bbox_inches='tight')
plt.show()

In [ ]:
# %matplotlib widget
from matplotlib.widgets import Button, Slider
L_init = 35.7
delta_init = 96.4

fig, ax = plt.subplots(1, 1, figsize=np.array([3.3*1.5, 2.2])*1.3)
# adjust the main plot to make room for the sliders
fig.subplots_adjust(left=0.25, bottom=0.25)

# Make a horizontal slider to control the frequency.
axfreq = fig.add_axes([0.25, 0.1, 0.65, 0.03])
L_slider = Slider(
    ax=axfreq,
    label='L dist',
    valmin=0.1,
    valmax=50,
    valinit=35.7,
)

# Make a vertically oriented slider to control the amplitude
axamp = fig.add_axes([0.1, 0.25, 0.0225, 0.63])
delta_slider = Slider(
    ax=axamp,
    label="Angle offset",
    valmin=-10,
    valmax=200,
    valinit=delta_init,
    orientation="vertical"
)

# Load reference data
ref_data = glob.glob('reference_data\\*.csv')
print(f"Found {len(ref_data)} reference data files!")
ref_2theta, ref_intensity = np.genfromtxt(ref_data[0], skip_header=0, unpack=True, delimiter=',', dtype=np.float64)

# Normalize to max=1
ref_intensity = ref_intensity / np.max(ref_intensity)
print(np.max(ref_intensity))
# Normalize to max=1
s_y_norm = subtracted_y / np.nanmax(subtracted_y)
b_yerr_norm = subtracted_b_yerr / np.nanmax(subtracted_y)
s_y_norm2 = subtracted_y2 / np.nanmax(subtracted_y)

ax.plot(ref_2theta, ref_intensity-0.2, 'g-', linewidth=0.84, zorder=4, alpha=1, label='Theory')
line1, = ax.plot(mm_to_two_theta(subtracted_x, d=30) + 85, s_y_norm, 'r-', linewidth=0.8, zorder=2, alpha=0.8, label='Background Subtracted Laue')
line2, = ax.plot(mm_to_two_theta(subtracted_x, d=30) + 85, s_y_norm2, 'b-', linewidth=0.8, zorder=2, alpha=0.8, label='Background Subtracted Laue')

# The function to be called anytime a slider's value changes
def update(val):
    line1.set_xdata(mm_to_two_theta(subtracted_x, d=L_slider.val) + delta_slider.val)
    line2.set_xdata(mm_to_two_theta(subtracted_x, d=L_slider.val) + delta_slider.val)
    fig.canvas.draw_idle()

# register the update function with each slider
L_slider.on_changed(update)
delta_slider.on_changed(update)

# Create a `matplotlib.widgets.Button` to reset the sliders to initial values.
resetax = fig.add_axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', hovercolor='0.975')

def reset(event):
    freq_slider.reset()
    amp_slider.reset()
button.on_clicked(reset)

ax.set(xlabel=r"$2\theta$ [deg]", ylabel='Intensity [a.u.]', ylim=(-0.3, 1.1))
plt.legend(framealpha=0)
hp.despine()
# plt.show()